In [1]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from readers.base import NeuralynxReader

In [2]:
tf.config.list_physical_devices('GPU')

2022-03-31 10:13:12.400172: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-31 10:13:12.442500: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-31 10:13:12.442653: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [26]:
rd = NeuralynxReader('/media/sil2/Lizard/lorenz_data/2019-11-17_17-43-33', channel=32, lowpass=50, decimate_q=10, wavelet='morl', window=1.2, overlap=0.5,
                     use_multiprocessing=False, use_slow_cycles=False, is_flat_cwt=False)
V, _, _ = rd.read_segmented(i_start=0, i_stop=int(rd.fs * 60))
V.shape

Buffered Matrix size: (4, 768)
Number of segments after buffering: 4
start run of segment function...
scales used: [16, 21, 26, 31, 36, 41, 46, 56, 76, 106, 171, 521]
start creation of sig_df...


  0%|          | 0/4 [00:00<?, ?it/s]

(4, 768, 12)

In [18]:
x = np.array([[1,2,3],[3,4,5]])
x = np.dstack([x,x,x,x])
print(x.shape)
for xx in np.rollaxis(x,2):
    print(xx.shape)

(2, 3, 4)
(2, 3)
(2, 3)
(2, 3)
(2, 3)


In [ ]:
def make_gen_model(latent_dim, n_inputs=768):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.ConvLSTM2D(32, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
    tf.keras.layers.UpSampling2D(size=(2, 2), data_format=None, interpolation="nearest")

In [ ]:
def make_gen_model(latent_dim, n_outputs=256):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(32, activation='relu', kernel_initializer='he_uniform', input_dim=latent_dim))
    model.add(tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.Dense(n_outputs, activation='linear'))
    return model

def make_disc_model(n_inputs=256):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(tf.keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.Dense(32, activation='relu', kernel_initializer='he_uniform'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def make_gan_model(disc_model, gen_modl):
    disc_model.trainable = False
    model = tf.keras.Sequential()
    model.add(gen_modl)
    model.add(disc_model)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

def train(disc_model, gen_model, gan_model, epochs, latent_dim = 5):
    batch_size = 128
    half_batch = int(batch_size / 2)
    for i in range(epochs):
    x_real, y_real = generate_real_sample(half_batch)
    x_fake, y_fake = generate_fake_sample(gen_model, latent_dim, half_batch)
    disc_model.train_on_batch(x_real, y_real)
    disc_model.train_on_batch(x_fake, y_fake)
    x_gan = generate_latent_points(latent_dim, batch_size)
    y_gan = np.ones((batch_size, 1))
    gan_model.train_on_batch(x_gan, y_gan)